##### 1.torch.utils.data 的学习

> 前面提到的 torchvision.datasets 实际上是基于 torch 的 数据加载与批处理框架 torch.utils.data 进行的二次封装，其中最核心的模块就是 torch.utils.data，它定义了 torch 中数据集表示、采样和批量加载的通用接口：

- 1）Dataset：数据集定义格式的基类

- 2）DataLoader：用于将 Dataset 封装成一个可迭代对象，实现了数据的批量加载、打乱、多线程

- 3）Subset：实现从现有的数据集里面提取子集，通常用作验证的划分

- 4）ConcatDataset 实现了多个数据集的拼接，前提是这些数据集的样本格式一致（\_\_getitem\_\_ 返回的结构相同）

- 5）还有 Tensor，常用于小型实验，用 Tensor 直接打包数据集，random_split 随即划分数据集

- 6）之前也讲过想要继承 Dataset 类的话必须实现两个方法，\_\_getitem\_\_ 和 \_\_len\_\_，torchvision.datasets 提供的类，本身就是 torch.utils.data.Dataset 的子类，所以可以直接被 DataLoader 使用，自己的数据集只引入 torchvision.datasets 是完全不够的，因为没办法生成 Dataset 类型的数据集，其只能用于加载官方的数据集直接使用，要是自己创建的数据集，需要手动实现上面说的类和类里面必须实现的方法

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset, TensorDataset
from tqdm import tqdm
import os
import torch
from PIL import Image
import torchvision.transforms as T

class MNIST(Dataset):
    def __init__(self, root, transform=None, preload=False):
        self.samples = []
        self.transform = transform
        self.preload = preload
        self.data = []
        classes = sorted(os.listdir(root))
        for label, cls in enumerate(tqdm(classes, desc="processing", ascii=True)):
            folder = os.path.join(root, cls)
            for img in os.listdir(folder):
                path = os.path.join(folder, img)
                self.samples.append((path, label))
        if self.preload:
            for path, label in self.samples:
                img = Image.open(path).convert("L")
                if self.transform:
                    img = self.transform(img)
                self.data.append((img, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        if self.preload:
            return self.data[index]
        path, label = self.samples[index]
        img = Image.open(path).convert("L") 
        if self.transform:
            img = self.transform(img)
        return img, label
transform = T.ToTensor()
dataset = MNIST("./mnist_test_torch", transform=transform, preload=True)
print(len(dataset))

# 验证 Subset
print("----------Subset:")
subset = Subset(dataset, indices=range(100))
print(len(subset))
image, label = subset[0]
print(image.shape, label)

# 验证 ConcatDataset
print("----------ConcatDataset:")
combined = ConcatDataset([dataset, dataset])
print(len(combined))
image, label = combined[0]
print(image.shape, label)

# 验证 TensorDataset
print("----------TensorDataset:")
ipt = torch.randn(1000, 1, 28, 28)
labels = torch.randint(0, 10, (1000,))
tensordataset = TensorDataset(ipt, labels)
print(len(tensordataset))
image, label = tensordataset[0]
print(image.shape, label)

# DataLoader 包含的参数主要包含下面几个：
# batch_size 每个批次的样本数量
# shuffle 是否打乱数据
# num_workers 加载数据的线程数
# drop_last 最后一个 batch 不足 batch_size 大小，是否丢弃

print("--------------------------------------------")
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for imgs, labels in loader: 
    # 执行这句话的时候 __getitem__ 才会执行，相当于进行下面的操作
    # batch_data = []
    # for idx in [i1, i2, ..., i32]:
    #     sample = dataset.__getitem__(idx)
    #     batch_data.append(sample)
    # 每张图片都要执行一边 getitem，设计图像的解码，打开，转换等操作，本质上是很慢的，它存在的意义就是有人需要对图像进行增强，一张图片可能被改编成很多种的可能，直接存储静态的文件实现不了这一点
    print(imgs.shape, labels.shape)
    break

processing: 100%|##########| 10/10 [00:00<00:00, 589.35it/s]


10000
----------Subset:
100
torch.Size([1, 28, 28]) 0
----------ConcatDataset:
20000
torch.Size([1, 28, 28]) 0
----------TensorDataset:
1000
torch.Size([1, 28, 28]) tensor(2)
--------------------------------------------
torch.Size([32, 1, 28, 28]) torch.Size([32])


##### 2.torch.utils.tensorboard、torch.utils.cpp_extension 和其他小众的 tensor.utils 内的包的学习

> 除去上面说的最重要的 torch.utils.dataset 另外比较重要的就是 torch.utils.tensorboard，这个接口能实现训练的可视化，但是使用 jupyter 的话本身就能实现可视化

- 1）代码里 from torch.utils.tensorboard import SummaryWriter 能正常导入，但命令行运行 tensorboard --version 提示 “命令不存在”

- 2）torch 自带的 TensorBoard 接口是可用的，但独立的 TensorBoard 可执行程序没有安装，两者是独立的，一个是负责存储，一个是网站打开查看

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("logs/exp1")
for i in range(100, 0, -1):
    epoch = i / 10
    loss = i
    writer.add_scalar(tag="loss/train", scalar_value=loss, global_step=epoch)
writer.close()

# tensorboard --logdir=logs 查看

> 还有就是 torch.utils.cpp_extension，允许在 python 里面编译并调用自定义的 C++ 或者 CUDA 算子，写高性能算子或者自定义后端的时候能用到

In [3]:
from torch.utils.cpp_extension import load

# 应该用不到，不做深入研究
# module = load(name="my_extension", sources=["my_op.cpp", "my_op_kernel.cu"])

> 除此上面的哪些模块之外，还包含一些其他小众模块：

- 1）torch.utils.benchmark：基准测试，简单说就是一个更专业版的 time.time()，能测算显存与 GPU 时间

- 2）torch.utils.dlpack：与其他框架共享张量（Numpy/TensorFlow/CuPy），当需要 torch、CuPy、TensorFlow 混用时，写底层算子、异构计算框架时会用到，不做演示

- 3）torch.utils.checkpoint：以计算换显存，在反向传播时重新计算部分前向结果，减少中间激活存储，非常适合大模型训练（Transformer、ViT、GPT类模型），其思想是：

- - 3.1）不保存中间结果，而是在反向传播时重新计算一部分前向传播，这样就可以节省显存（少存激活），代价是计算量多一点（要多算几次前向）

> 先知道有这个东西吧，需要的时候再来学习

In [7]:
import torch
import torch.utils.benchmark as benchmark

x = torch.randn(1000, 1000, device='cuda')
y = torch.randn(1000, 1000, device='cuda')


t = benchmark.Timer(
    stmt="torch.mm(x, y)", # 要执行的语句主体，也就是测试主体
    # setup=None, # 测试前要执行的初始化代码
    globals={"x": x, "y": y}, # 传入外部变量环境，替代 setup 字符串方式
    # label=None, # 可选标签，用于结果分组显示
    num_threads=torch.get_num_threads()
)

m = t.timeit(1000) # 执行 stmt n 次，返回 Measurement 对象

print(m, m.mean, m.median, m.times) # 平均数、中位数、所有测量列表

torch.mm(x, y)
  139.22 us
  1 measurement, 1000 runs , 8 threads 0.0001392244000453502 0.0001392244000453502 [0.0001392244000453502]


> 防止梯度爆炸的一些函数

- 1）torch.nn.utils.clip_grad_value_，限制每个梯度的绝对值在某个范围内（逐元素裁剪）

- 2）torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)，限制整体的范数不超过阈值，超过的话整体进行缩放（整体裁剪）